LIEN
https://developer.riotgames.com/
https://jsonformatter.org/json-viewer
INFO
20 requests every 1 seconds(s)
100 requests every 2 minutes(s)
mastery champion by psuedo : https://euw1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/UBmXL4pcbdSP8_hcig01ySFtIQBnknrEAjvBKjckC-dkgUghttps://euw1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/UBmXL4pcbdSP8_hcig01ySFtIQBnknrEAjvBKjckC-dkgUg

In [1]:
import requests
import pandas as pd

#Correspondance id_ spell/champion
spell_id = {21: "Barrier", 1: "Cleanse", 14: "Ignite", 3: "Exhaust", 4: "Flash", 6: "Ghost", 7: "Heal", 13: "Clarity", 30: "To the King!", 31: "Poro Toss", 11: "Smite", 39: "Mark", 32: "Mark", 12: "Teleport", 54: "Placeholder", 55: "Placeholder and Attack-Smite"}
champion_id = {266: "Aatrox", 103: "Ahri", 84: "Akali", 166: "Akshan", 12: "Alistar", 32: "Amumu", 34: "Anivia", 1: "Annie", 523: "Aphelios", 22: "Ashe", 136: "Aurelion Sol", 268: "Azir", 432: "Bard", 53: "Blitzcrank", 63: "Brand", 201: "Braum", 51: "Caitlyn", 164: "Camille", 69: "Cassiopeia", 31: "Cho Gath", 42: "Corki", 122: "Darius", 131: "Diana", 119: "Draven", 36: "Dr. Mundo", 245: "Ekko", 60: "Elise", 28: "Evelynn", 81: "Ezreal", 9: "Fiddlesticks", 114: "Fiora", 105: "Fizz", 3: "Galio", 41: "Gangplank", 86: "Garen", 150: "Gnar", 79: "Gragas", 104: "Graves", 887: "Gwen", 120: "Hecarim", 74: "Heimerdinger", 420: "Illaoi", 39: "Irelia", 427: "Ivern", 40: "Janna", 59: "Jarvan IV", 24: "Jax", 126: "Jayce", 202: "Jhin", 222: "Jinx", 145: "Kai Sa", 429: "Kalista", 43: "Karma", 30: "Karthus", 38: "Kassadin", 55: "Katarina", 10: "Kayle", 141: "Kayn", 85: "Kennen", 121: "Kha Zix", 203: "Kindred", 240: "Kled", 96: "Kog Maw", 7: "LeBlanc", 64: "Lee Sin", 89: "Leona", 876: "Lillia", 127: "Lissandra", 236: "Lucian", 117: "Lulu", 99: "Lux", 54: "Malphite", 90: "Malzahar", 57: "Maokai", 11: "Master Yi", 21: "Miss Fortune", 62: "Wukong", 82: "Mordekaiser", 25: "Morgana", 267: "Nami", 75: "Nasus", 111: "Nautilus", 518: "Neeko", 76: "Nidalee", 56: "Nocturne", 20: "Nunu & Willump", 2: "Olaf", 61: "Orianna", 516: "Ornn", 80: "Pantheon", 78: "Poppy", 555: "Pyke", 246: "Qiyana", 133: "Quinn", 497: "Rakan", 33: "Rammus", 421: "Rek Sai", 526: "Rell", 888: "Renata Glasc", 58: "Renekton", 107: "Rengar", 92: "Riven", 68: "Rumble", 13: "Ryze", 360: "Samira", 113: "Sejuani", 235: "Senna", 147: "Seraphine", 875: "Sett", 35: "Shaco", 98: "Shen", 102: "Shyvana", 27: "Singed", 14: "Sion", 15: "Sivir", 72: "Skarner", 37: "Sona", 16: "Soraka", 50: "Swain", 517: "Sylas", 134: "Syndra", 223: "Tahm Kench", 163: "Taliyah", 91: "Talon", 44: "Taric", 17: "Teemo", 412: "Thresh", 18: "Tristana", 48: "Trundle", 23: "Tryndamere", 4: "Twisted Fate", 29: "Twitch", 77: "Udyr", 6: "Urgot", 110: "Varus", 67: "Vayne", 45: "Veigar", 161: "Vel Koz", 711: "Vex", 254: "Vi", 234: "Viego", 112: "Viktor", 8: "Vladimir", 106: "Volibear", 19: "Warwick", 498: "Xayah", 101: "Xerath", 5: "Xin Zhao", 157: "Yasuo", 777: "Yone", 83: "Yorick", 350: "Yuumi", 154: "Zac", 238: "Zed", 221: "Zeri", 115: "Ziggs", 26: "Zilean", 142: "Zoe", 143: "Zyra"}


In [2]:
def historique(pseudo, rgn, region, nb_game, RAPI):
    #Récupération des informations du joueur pseudo
    pseudo_link = "https://{}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{}?api_key={}".format(rgn, pseudo, RAPI)
    pseudo = requests.get(pseudo_link).json()

    print ("Historique de:", pseudo['name'], "\n")

    #Récupération de nb_game games dans mon historique
    matches_link = "https://{}.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?api_key={}&start=0&count={}".format(region, pseudo['puuid'], RAPI, nb_game)
    matches = requests.get(matches_link).json()

    #Boucle sur chaque game, pour récupérer les informations
    matches_array=[]
    for match in matches:
        match_detail_link = "https://{}.api.riotgames.com/lol/match/v5/matches/{}?api_key={}".format(region, match, RAPI)
        match_detail = requests.get(match_detail_link).json()

    #Boucle sur chaque joueur, pour récupérer leurs informations
        players = []
        for player_row in match_detail['info']['participants']:
            player = {}
            player['pseudo'] = player_row['summonerName']
            player['champion'] = player_row['championName']
            player['win'] = player_row['win']
            player['kills'] = player_row['kills']
            player['deaths'] = player_row['deaths']
            player['assists'] = player_row['assists']
            player['csing'] = player_row['totalMinionsKilled']
            player['fb'] = player_row['firstBloodKill']
            players.append(player)

        df_players = pd.DataFrame(players)
        matches_array.append(df_players)

    #Concatenation des games une à une, dans un grand Dataframe
    df = pd.concat(matches_array).reset_index()
    return df

In [3]:
def ingame(pseudo_live, rgn, RAPI):
    #Récupération des information du joueur pseudo_test
    pseudo_live_link = "https://{}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{}?api_key={}".format(rgn, pseudo_live, RAPI)
    pseudo_live = requests.get(pseudo_live_link).json()

    print("Game de: ", pseudo_live['name'], "\n")

    #Récupération de la game en cours
    live_match_link = "https://{}.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{}?api_key={}".format(rgn, pseudo_live['id'], RAPI)
    live_match = requests.get(live_match_link).json()

    players_ingame = []
    #Si le joueur pseudo_test n'est pas en game print, sinon check le mode de jeu puis boucle sur tout les joueurs de la game
    if 'status' in live_match.keys():
        print("Player is not in game")
    elif live_match['gameMode'] == "CLASSIC":
        for player_row in live_match['participants']:
            player = {}
            player['pseudo'] = player_row['summonerName']
            player_rank = requests.get("https://{}.api.riotgames.com/lol/league/v4/entries/by-summoner/{}?api_key={}".format(rgn, player_row['summonerId'], RAPI)).json()
            if len(player_rank) == 0:
                player['rank'] = "unranked"
            elif player_rank[0]['queueType'] == "RANKED_SOLO_5x5":
                player['rank'] = player_rank[0]['tier'] + " " + player_rank[0]['rank']
            else:
                player['rank'] = player_rank[1]['tier'] + " " + player_rank[1]['rank']
            player['champion'] = champion_id[player_row['championId']]
            player['spell1'] = spell_id[player_row['spell1Id']]
            player['spell2'] = spell_id[player_row['spell2Id']]
            if player_row['teamId'] == 100:
                player['team'] = "blue"
            else:
                player['team'] = "red"
            players_ingame.append(player)
    elif live_match['gameMode'] == "ARAM":
        for player_row in live_match['participants']:
            player = {}
            player['pseudo'] = player_row['summonerName']
            player_mmr = requests.get("https://euw.whatismymmr.com/api/v1/summoner?name={}".format(player_row['summonerName'])).json()
            if 'error' in player_mmr.keys():
                player['mmr'] = "N/A"
            else:
                player['mmr'] = str(player_mmr['ARAM']['avg']) + " +- " + str(player_mmr['ARAM']['err'])
            player_rank = requests.get("https://{}.api.riotgames.com/lol/league/v4/entries/by-summoner/{}?api_key={}".format(rgn, player_row['summonerId'], RAPI)).json()
            if not player_rank:
                player['rank'] = "unranked"
            else:
                for i in range(len(player_rank)):
                    if player_rank[i]['queueType'] == "RANKED_SOLO_5x5":
                        player['rank'] = player_rank[i]['tier'] + " " + player_rank[i]['rank']
            player['champion'] = champion_id[player_row['championId']]
            player['spell1'] = spell_id[player_row['spell1Id']]
            player['spell2'] = spell_id[player_row['spell2Id']]
            if player_row['teamId'] == 100:
                player['team'] = "blue"
            else:
                player['team'] = "red"
            players_ingame.append(player)

    #Dataframe
    df = pd.DataFrame(players_ingame)
    return df

In [4]:
#Variables
pseudo = "jsob"
RAPI = 'RGAPI-22e6688c-5e89-4d92-a9fd-0d9a52b069c9'
rgn = 'euw1'
region = "europe"
nb_game = 3

In [5]:
histo = historique(pseudo, rgn, region, nb_game, RAPI)
df_histo = pd.DataFrame(histo)
df_histo

Historique de: JSOB 



,index,pseudo,champion,win,kills,deaths,assists,csing,fb
0,0,NikiSpring,Yorick,False,4,9,5,251,False
1,1,Allvar,Hecarim,False,14,11,11,32,True
2,2,Allo Uix,Swain,False,1,10,9,152,False
3,3,Katardarubet,Xayah,False,15,9,5,258,False
4,4,JSOB,Nautilus,False,1,12,13,35,False
5,5,Lamb of Farmland,Illaoi,True,15,7,11,295,False
6,6,Calvitie Précoce,Ekko,True,20,3,11,56,False
7,7,L1 Sociologie,Qiyana,True,6,10,12,172,False
8,8,VeKtik,Sivir,True,6,10,15,222,False
9,9,Captaın Jack,Bard,True,4,6,23,32,False


In [6]:
live = ingame("chenzelot", rgn, RAPI)
df_live = pd.DataFrame(live)
df_live

Game de:  chenzelot 



,pseudo,rank,champion,spell1,spell2,team
0,RobinV2testacc,GRANDMASTER I,Lillia,Flash,Smite,blue
1,LFT Retnyam,CHALLENGER I,Jax,Teleport,Flash,blue
2,5R LUKA1,CHALLENGER I,Lucian,Exhaust,Flash,blue
3,ottıe,GRANDMASTER I,Xayah,Flash,Heal,blue
4,schifosoèmerdoso,CHALLENGER I,Karma,Ignite,Flash,blue
5,chenzelot,GRANDMASTER I,Irelia,Flash,Teleport,red
6,HiY,GRANDMASTER I,Nautilus,Flash,Ignite,red
7,KC Rekkles,CHALLENGER I,Zeri,Flash,Heal,red
8,KHK TLamp,CHALLENGER I,Xerath,Flash,Teleport,red
9,to eternally rot,CHALLENGER I,Zed,Smite,Flash,red


In [7]:
live2 = ingame("mavs", rgn, RAPI)
df_live2 = pd.DataFrame(live2)
df_live2

Game de:  Mavs 



,pseudo,mmr,rank,champion,spell1,spell2,team
0,jorgitoman96,N/A,unranked,Darius,Mark,Flash,blue
1,PsychoLupin,N/A,NaN,Garen,Mark,Flash,blue
2,aImondiga,2382 +- 99,PLATINUM III,Vex,Mark,Flash,blue
3,Risi,2842 +- 39,NaN,Xerath,Flash,Exhaust,blue
4,Chickenwing2020,3453 +- 159,NaN,Nasus,Mark,Flash,blue
5,Mavs,N/A,NaN,Yuumi,Exhaust,Ignite,red
6,verybiglama,3492 +- 171,unranked,Katarina,Mark,Flash,red
7,BØNIN,3235 +- 73,DIAMOND IV,Samira,Mark,Flash,red
8,DasBlaueBlue,3180 +- 96,DIAMOND II,Lulu,Flash,Exhaust,red
9,Zwottr0nin,3354 +- 209,GOLD I,Zeri,Flash,Cleanse,red
